In [4]:
import torch
from multi_scale_attention import MultiScaleAttention

In [5]:
B = 4
C = 3
H = 480
W = 640
device = 'cpu'
ms_attention = MultiScaleAttention(32, num_heads=4, sr_ratio=8, local_region_shape=[5,10,20,40])
ms_attention = ms_attention.to(device)
# ms_attention = nn.DataParallel(ms_attention, device_ids = [0,1])
# ms_attention.to(f'cuda:{ms_attention.device_ids[0]}', non_blocking=True)

f = torch.randn(B, 19200, 32).to(device)
print(f'input to multiScaleAttention:{f.shape}')
y = ms_attention(f, 120, 160)
print('output: ',y.shape)

input to multiScaleAttention:torch.Size([4, 19200, 32])
output:  torch.Size([4, 19200, 32])


In [14]:
print(H//4*W//4*C*B*16)
print(4*19200*32)

print(120*160*32)

3686400
2457600
614400
